In [72]:
import os
print(os.path.abspath('.'))
os.popen('mkdir write')
os.popen('mkdir figures')

/data1/xfwang/Jupyter_notebook/Velocity/MDSC


In [73]:
import sys
import numpy as np
from scipy.stats import norm
from scipy.spatial.distance import pdist, squareform
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import loompy
import velocyto as vcy
import pandas as pd

from IPython.core.display import display, HTML
import logging
logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

2020-11-17 00:03:44,176 - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [74]:
## 读取loom文件的方法
dirr="./datasource/"
# loomT = vcy.VelocytoLoom(dirr+"CRCSZ04T_out.loom")
dsH119WT = loompy.connect(dirr+"H119WT_out.loom")
# loomP = vcy.VelocytoLoom(dirr+"CRCSZ04P_out.loom")
dsH389WT = loompy.connect(dirr+"H389WT_out.loom")
# loomN = vcy.VelocytoLoom(dirr+"CRCSZ04N_out.loom")
dsH124KO = loompy.connect(dirr+"H124KO_out.loom")
dsH382jia390KO = loompy.connect(dirr+"H382jia390KO_out.loom")
## 输出每个loom文件中的细胞数
print(len(dsH119WT.ca['CellID']))
print(len(dsH389WT.ca['CellID']))
print(len(dsH124KO.ca['CellID']))
print(len(dsH382jia390KO.ca['CellID']))
cell_dsH119WT = pd.DataFrame(dsH119WT.ca['CellID'])
cell_dsH119WT.columns=['CellName']
len(cell_dsH119WT['CellName'].unique())

6732
7943
8432
7045


6732

In [5]:
# #### 第二种方法：合并多个样品的loom文件（推荐）
## 需安装loompy版本为2.0.7才可使用
## 在命令行运行： ls ./*.loom |  sed "s:^:`pwd`/: " > loom.file.list
## 合并MDSC的四个loom文件并写为merged4.loom
files = []
inf = open("./datasource/loom.file.list", "r")
for line in inf:
    files.append(line.strip())
inf.close()
dirr = "./write/"
loompy.combine(files, dirr+"MDSC_merged4.loom", key="Accession")

2020-11-16 11:53:27,493 - WARNING - ╭── 'batch_scan_layers' is deprecated. Use 'scan' instead
2020-11-16 11:53:27,494 - WARNING - ╰──> at /home/hou/anaconda3/lib/python3.6/site-packages/loompy/loompy.py, line 465
2020-11-16 11:54:08,514 - WARNING - ╭── 'batch_scan_layers' is deprecated. Use 'scan' instead
2020-11-16 11:54:08,515 - WARNING - ╰──> at /home/hou/anaconda3/lib/python3.6/site-packages/loompy/loompy.py, line 465
2020-11-16 11:54:43,392 - WARNING - ╭── 'batch_scan_layers' is deprecated. Use 'scan' instead
2020-11-16 11:54:43,393 - WARNING - ╰──> at /home/hou/anaconda3/lib/python3.6/site-packages/loompy/loompy.py, line 465


In [6]:
ds = loompy.connect(dirr+"MDSC_merged4.loom")
## 输出每个loom文件中的细胞数
print(len(ds.ca['CellID']))
cell_ds = pd.DataFrame(ds.ca['CellID'])
cell_ds.columns=['CellName']
len(cell_ds['CellName'].unique())

30152


30152

In [10]:
vlm = vcy.VelocytoLoom(dirr+"MDSC_merged4.loom")

In [11]:
vlm.ca

{'CellID': array(['H119WT_out:AAAGGGCCAGAACCGAx', 'H119WT_out:AAACGCTAGGCCTAGAx',
        'H119WT_out:AAAGGATTCAAGGTGGx', ...,
        'H389WT_out:TTTGGTTAGACTCTTGx', 'H389WT_out:TTTGACTTCGCGTTTCx',
        'H389WT_out:TTTGTTGAGACTGGGTx'], dtype=object),
 'Clusters': array([ 2,  5, 10, ...,  0,  0,  0]),
 '_X': array([-31.861305 ,   2.6219785,  -9.111278 , ..., -19.501108 ,
        -22.226475 , -26.800665 ], dtype=float32),
 '_Y': array([ 22.487562, -45.667614,  28.579712, ...,  22.811373,  32.651466,
         14.871033], dtype=float32)}

In [17]:
annotation = pd.read_csv("./annotation.csv",index_col=0)
print(annotation.shape)
annotation.iloc[-3:,:]

(11516, 10)


,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rp,group,celltype,Lcelltype,seurat_clusters,RNA_snn_res.1.5
H382jia390KOTTACTGTTCAGTCTTT,H382jia390KO,5557,1626,10.185352,24.221702,KO,ILC2_Il5,ILC,24,24
H382jia390KOTTCGGTCAGATTGTGA,H382jia390KO,4092,1455,5.303030,30.840665,KO,ILC2_Il5,ILC,24,24
H382jia390KOTTTACTGTCATGCCCT,H382jia390KO,3584,1396,11.356027,17.717634,KO,ILC2_Il5,ILC,24,24


In [66]:
## 提取所需的子细胞集
x = vlm.ca["CellID"]
# print(x)
y = {}
for i in x:
    y[i] = i.replace("_out:","").replace("x","") 
# print(y)
subName = []
# subdict = {}
for key,value in y.items():
#     print(j)
    if value in annotation._stat_axis.values.tolist():#rownames
        subName.append(key)
#         subdict[key]=value

In [67]:
print(len(subName))
print(len(subdict))
print(len(np.unique(subName)))# 检查是否有重复的细胞名称
print(type(subName))
print(subName[0:3])

11496
11496
11496
<class 'list'>
['H119WT_out:AACACACTCGCCTCTAx', 'H119WT_out:AAACGAAGTCATCGGCx', 'H119WT_out:AAACGAACACTGTGATx']


In [64]:
## 读入和写出最终需要分析的细胞名称的txt文件
dirr = "./write/"
f=open(dirr+"FinalCell_velocyto.txt","w")
# f.writelines(subName)
f.write(str(subName))
f.close()

In [68]:
import copy
vlm_0=copy.deepcopy(vlm)

In [69]:
#### 根据已知的细胞名称，取出loom文件的子集
print(len(subName))
print(len(vlm.ca["CellID"]))

targetCell = set(subName)
mask = []
for e in vlm.ca["CellID"]:
    if e in targetCell:
        mask.append(True)
    else:
        mask.append(False)
        
print(len(mask))
false = [e for e in mask if e == False]
print(len(false))
mask2 = np.array(mask)
print(len(mask2))
type(mask2)

### 过滤vlm对象，取子集
vlm.filter_cells(bool_array=mask2)

11496
30152
30152
18656
30152


In [70]:
## 写出hdf5文件进行保存
dirr = "./write/"
vlm.to_hdf5(dirr+'MDSC_immuneCell_R.hdf5')
## 读取hdf5文件进行分析
# vlm_sub = vcy.load_velocyto_hdf5('MDSC_immuneCell_R.hdf5')

In [71]:
vlm.S.shape

(31053, 11496)